In [308]:
import numpy as np

import jax
import jax.numpy as jnp
from jax import jit, grad, vmap

import pickle as pkl

from alive_progress import alive_bar

parallel_scan = jax.lax.associative_scan

## Make a single LRU

In [309]:
def binary_operator_diag(element_i, element_j):
    a_i, bu_i = element_i
    a_j, bu_j = element_j

    return a_j * a_i, a_j * bu_i + bu_j


def init_lru_parameters(N, H, r_min = 0.0, r_max = 1, max_phase = 6.28):
    # N: state dimension, H: model dimension
    # Initialization of Lambda is complex valued distributed uniformly on ring
    # between r_min and r_max, with phase in [0, max_phase].

    u1 = np.random.uniform(size = (N,))
    u2 = np.random.uniform(size = (N,))

    nu_log = np.log(-0.5*np.log(u1*(r_max**2-r_min**2) + r_min**2))
    theta_log = np.log(max_phase*u2)

    # Glorot initialized Input/Output projection matrices
    B_re = np.random.normal(size=(N,H))/np.sqrt(2*H)
    B_im = np.random.normal(size=(N,H))/np.sqrt(2*H)
    C_re = np.random.normal(size=(H,N))/np.sqrt(N)
    C_im = np.random.normal(size=(H,N))/np.sqrt(N)
    D = np.random.normal(size=(H,))

    # Normalization
    diag_lambda = np.exp(-np.exp(nu_log) + 1j*np.exp(theta_log))
    gamma_log = np.log(np.sqrt(1-np.abs(diag_lambda)**2))

    return nu_log, theta_log, B_re, B_im, C_re, C_im, D, gamma_log


def forward_LRU(lru_parameters, input_sequence):
    # Unpack the LRU parameters
    nu_log, theta_log, B_re, B_im, C_re, C_im, D, gamma_log = lru_parameters

    # Initialize the hidden state
    Lambda = jnp.exp(-jnp.exp(nu_log) + 1j*jnp.exp(theta_log))
    B_norm = (B_re + 1j*B_im) * jnp.expand_dims(jnp.exp(gamma_log), axis=-1)
    C = C_re + 1j*C_im

    Lambda_elements = jnp.repeat(Lambda[None, ...], input_sequence.shape[0], axis=0)

    Bu_elements = jax.vmap(lambda u: B_norm @ u)(input_sequence)
    elements = (Lambda_elements, Bu_elements)
    _, inner_states = parallel_scan(binary_operator_diag, elements) # all x_k
    y = jax.vmap(lambda x, u: (C @ x).real + D * u)(inner_states, input_sequence)


    return y

def loss_fn(lru_parameters, input_sequence, target_sequence):
    y = forward_LRU(lru_parameters, input_sequence)
    return jnp.mean((y - target_sequence)**2)


def update(lru_parameters, input_sequence, target_sequence):
    return  grad(loss_fn)(lru_parameters, input_sequence, target_sequence)

In [310]:
# Create the MLP encoder

In [311]:
def init_mlp_parameters(layers):
    # Initialize the MLP parameters
    parameters = []
    for i in range(len(layers)-1):
        W = np.random.normal(size=(layers[i], layers[i+1]))/np.sqrt(layers[i])
        b = np.zeros((layers[i+1],))
        parameters.append((W, b))

    return parameters

@jit
def forward_mlp(mlp_parameters, input, activation_function = jnp.tanh):
    # Forward pass of the MLP
    
    x = input

    for W, b in mlp_parameters:
        x = x @ W + b
        x = activation_function(x)

    return x

def forward_mlp_linear_layer(mlp_parameters, input, activation_function = jnp.tanh):
    
    x = input

    # Only apply the MLP up to the second last layer
    for W, b in mlp_parameters[:-1]:
        x = x @ W + b
        x = activation_function(x)

    # Apply the last layer without activation function
    W, b = mlp_parameters[-1]
    x = x @ W + b

    # Use the softmax function on the last layer
    x = jax.nn.softmax(x)

    return x


In [312]:
def max_pooling(sequence_to_pool):
    return jnp.max(sequence_to_pool, axis=0)

def mean_pooling(sequence_to_pool):
    return jnp.mean(sequence_to_pool, axis=0)

def sum_pooling(sequence_to_pool):
    return jnp.sum(sequence_to_pool, axis=0)

In [ ]:
# Create the model
Linear_encoder_parameter  = init_mlp_parameters([1,3,5])
seconday_parameters = init_mlp_parameters([5,5,5])
LRU = init_lru_parameters(5, 5)
Linear_decoder_parameter = init_mlp_parameters([5,3,2])

model_parameters = (Linear_encoder_parameter, seconday_parameters, LRU, Linear_decoder_parameter)

def model_forward(input_sequence, parameters):
    Linear_encoder_parameter, seconday_parameters, LRU, Linear_decoder_parameter = parameters

    x = forward_mlp(Linear_encoder_parameter, input_sequence)
    x = forward_LRU(LRU, x)
    x = forward_mlp(seconday_parameters, x)
    x = max_pooling(x)
    x = forward_mlp_linear_layer(Linear_decoder_parameter, x)

    return x

def one_hot(x, k, dtype=jnp.float32):
  """Create a one-hot encoding of x of size k."""
  return jnp.array(x[:, None] == jnp.arange(k), dtype)

def model_loss(input_sequence, target_sequence, parameters):
    y = model_forward(input_sequence, parameters)

    # Binary cross entropy loss
    return -jnp.mean(target_sequence * jnp.log(y) + (1-target_sequence) * jnp.log(1-y))

def model_grad(input_sequence, target_sequence, parameters):
    return grad(model_loss)(input_sequence, target_sequence, parameters)

In [337]:
model_forward(jnp.array([[1.0], [2.0], [3.0]]), model_parameters)


Array([0.59122086, 0.4087791 ], dtype=float32)

In [338]:
# Prepare the dataset
waveforms = pkl.load(open("waveforms.pkl", "rb"))


sequences = np.concat((waveforms["waveform1"], waveforms["waveform2"]))
labels = np.concat((np.zeros(500),np.ones(500)))

# Shuffle the dataset
perm = np.random.permutation(1000)
sequences = sequences[perm]
labels = labels[perm]
labels = one_hot(labels, 2)

In [349]:
jax.jacobian(jax.jacobian(model_loss))(jnp.array([[1.0], [2.0], [3.0]]), 1, model_parameters)

c:\Users\David\Documents\ML_jax\.venv\lib\site-packages\jax\_src\lax\lax.py:3373: ComplexWarning: Casting complex values to real discards the imaginary part
  x_bar = _convert_element_type(x_bar, x.aval.dtype, x.aval.weak_type)


Array([[[[-5.8554634e-03],
         [ 2.5951770e-06],
         [-4.3208975e-06]]],


       [[[ 2.5951726e-06],
         [-4.0689006e-04],
         [-2.4504808e-07]]],


       [[[-4.3208984e-06],
         [-2.4504811e-07],
         [-5.2616673e-05]]]], dtype=float32)

In [320]:
epochs = 10
batchsize = 1

for i in range(epochs):

    for x, y in zip(sequences, labels):
        grad = model_grad(x.reshape((len(x), 1)), y, model_parameters)
        #model_parameters = [p - 0.01*g for p, g in zip(model_parameters, grad)]


TypeError: 'jaxlib.xla_extension.ArrayImpl' object is not callable

In [ ]:
jnp.array([[1.0], [2.0], [3.0]]).shape

(3, 1)

In [ ]:
# Define a scalar-valued function
def my_function(x):
    return jnp.sum(x ** 2)

# Use grad on the function
grad_fn = jax.grad(my_function)

# Evaluate gradient at a specific point
x = jnp.array([1.0, 2.0, 3.0])
gradient = grad_fn(x)
print("Gradient:", gradient)

<class 'jaxlib.xla_extension.ArrayImpl'>
Gradient: [2. 4. 6.]
